In [2]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
from copy import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import fnmatch
import csv
import pickle
import warnings
import torch
import scipy
from scipy import optimize
from scipy.optimize import leastsq
from io import StringIO
from scipy.signal import savgol_filter
from datetime import datetime

now = datetime.now()
d_string = now.strftime("%d.%m.%Y")



## Load predictions and training data

In [211]:
#train_energy = pd.read_csv('Data/Results/%(now)s/train_energy.csv' % {"now": d_string}, header=None, names=['energy'])
#train_time = pd.read_csv('Data/Results/%(now)s/train_time.csv' % {"now": d_string}, header=None, names=['time'])
#N_train = len(train_energy)

In [11]:
path_1 = r'Data/Results/%(now)s/' % {"now": d_string} # use your path
path_2 = r'xyp1p2_all_e30000_round3_*'
all_files = glob.glob(path_1 + path_2)
li = []

for filename in all_files:
    df = pd.read_csv(filename, delimiter=",", skiprows=1, header=None, usecols=[0,1,2,3,4])
    li.append(df)
frame = pd.concat(li, axis=0, ignore_index=True)
print(frame)

               0         1         2    3    4
0       0.033501  0.102166  0.102345  0.1  2.0
1      -0.325543  0.103743  0.103038  0.1  2.0
2       0.303909  0.102542  0.102371  0.1  2.0
3      -0.373005  0.540013  0.562950  1.0  0.6
4      -0.029276  0.102413  0.102360  0.1  2.0
...          ...       ...       ...  ...  ...
361635 -0.360869  0.104664  0.103547  0.1  2.0
361636  0.828239  0.103443  0.102477  0.1  2.0
361637 -0.580306  0.103470  0.102875  0.1  2.0
361638 -0.978961  0.103231  0.102538  0.1  2.0
361639 -0.213194  0.102526  0.102576  0.1  2.0

[361640 rows x 5 columns]


In [12]:
pseudo_x = frame.iloc[:,0]
pseudo_y = frame.iloc[:,1]
pseudo_pred = frame.iloc[:,2]
pseudo_time = frame.iloc[:,3]
pseudo_energy = frame.iloc[:,4]

## Load Chi2 training data

In [13]:
path_3 = r'chi_all_*' 

all_files_chi = glob.glob(path_1 + path_3)
chi2_array = []
chi2_index = []

for filename in all_files_chi:
    df = pd.read_csv(filename, delimiter=",", skiprows=1, header=None, usecols=[0,1])
    best_try = np.where(df[1] == min(df[1]))[0]
    chi2_array.append(min(df[1]))
    chi2_index.append(max(best_try))

## Load extrapolation data

In [43]:
path_4 = r'extrapolation_e30000_round3_*'
all_files_ex = glob.glob(path_1 + path_4)
lo=[]

for filename in all_files_ex:
    df = pd.read_csv(filename, delimiter=",", skiprows=1, header=None, usecols=[0,3], names=['dE', 'D'])
    lo.append(df)
extrapolation = pd.concat(lo, axis=0, ignore_index=True)


## Load interpolation data

In [52]:
path_5 = r'extrapolation_e30000_round2*.csv'
all_files_int = glob.glob(path_1 + path_5)
la = []
for filename in all_files_int:
    df = pd.read_csv(filename, delimiter=",", skiprows=1, header=None, usecols=[0,1,2,3], names=['dE', 'time', 'energy', 'D'])
    la.append(df)
interpolation = pd.concat(la, axis=0, ignore_index=True)
N_interpol = int(np.divide(len(interpolation), 20))



1662
